<a href="https://colab.research.google.com/github/suZanahoria287/Conversor-imagenes-a-PDF/blob/main/ConversorImgPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ⬇️ Instala libs
!pip install --quiet Pillow img2pdf pypdf

import io, os
from PIL import Image
import img2pdf
from pypdf import PdfReader
from google.colab import files

print("Selecciona todas las IMÁGENES (puedes seleccionar múltiples).")
uploaded = files.upload()  # selecciona varios archivos

# Filtra y ordena por nombre
valid_ext = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp')
imagenes = [n for n in uploaded.keys() if n.lower().endswith(valid_ext)]
imagenes.sort(key=lambda s: s.lower())

if not imagenes:
    raise RuntimeError("No se detectaron imágenes válidas.")

# Convierte cada imagen a bytes compatibles y arma el PDF en memoria
buffers = []
for name in imagenes:
    im = Image.open(name)

    # Normaliza modos problemáticos y transparencia
    if im.mode == "P":
        im = im.convert("RGBA")
    if im.mode in ("RGBA", "LA"):
        alpha = im.split()[-1]
        bg = Image.new("RGB", im.size, (255, 255, 255))
        bg.paste(im, mask=alpha)
        im = bg
    elif im.mode not in ("RGB", "L"):
        im = im.convert("RGB")

    # Guarda a bytes (JPEG para robustez)
    buf = io.BytesIO()
    # Calidad alta sin exagerar tamaño
    im.save(buf, format="JPEG", quality=95, optimize=True)
    buffers.append(buf.getvalue())

# Genera el PDF único
output_pdf = "imagenes_unidas.pdf"
pdf_bytes = img2pdf.convert(buffers)
with open(output_pdf, "wb") as f:
    f.write(pdf_bytes)

# Valida que el PDF sea legible y cuenta páginas
pages = len(PdfReader(output_pdf).pages)
print(f"✅ PDF generado correctamente con {pages} página(s): {output_pdf}")

# Descarga
files.download(output_pdf)
